Here to run the code and try things 

Wiii


In [2]:
import numpy as np
import fit_shape

In [ ]:
# Here we make a fit to the 12 CO surface of the disk J1615
# input order must be: path, name, d, vlsr, z0, psi, PA, inc, ...

dicti = fit_shape.fit_Gas('/Volumes/Transcend/J1615_2020/DataWork/Results/CO/',
                    'J1615_12CO_im_uv006_', 
                    157.68, 4744.61, 0.2471, 1.1814, 325.03, 46.883, mstar=1.183, x0=0, y0=0,
                    r_min=1.5, r_max=2.3, downsample=25, clip=3, nwalkers=50, nburnin=500,nsteps=1500, beam=False, just_results=True)
                    

In [ ]:
dicti

In [ ]:
mstar = dicti.get('mstar')
pa = dicti.get('PA')
inc = dicti.get('inc')
z0 = dicti.get('z0')
psi = dicti.get('psi')
vlsr = dicti.get('vlsr')/1000


In [32]:
mstar, pa, inc, z0, psi, vlsr =(1.2047920150052118, 325.15077530944075, 46.41703391829195, 0.2494024079038776, 1.211576566441788, 4.741979863714648)

(1.2047920150052118,
 325.15077530944075,
 46.41703391829195,
 0.2494024079038776,
 1.211576566441788,
 4.741979863714648)

In [14]:
vels_lev = np.array([3,3.7,4.4,5.1,5.8,6.5])

6

In [50]:
from Isovel import Isovel 

In [51]:
Isovel = Isovel(mstar, pa, inc, z0, psi, vlsr, vels_lev)

In [53]:
vel_thin, vel_front, vel_back = Isovel.calculate_vel()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [88]:
import Isovel_2 

In [89]:
R_max=300
v_max=5
nx=300
ny=300
southcloser=True
pix_to_au =1

mstar, pa, inc, z0, psi, vlsr =(1.2047920150052118, 325.15077530944075, 46.41703391829195,   
                                0.2494024079038776, 1.211576566441788, 4.741979863714648)
vels_lev = np.array([3,3.7,4.4,5.1,5.8,6.5])

vel_thin, vel_front, vel_back = Isovel_2.calculate_vel(mstar, pa, inc, z0, psi, vlsr, vels_lev)
print(vel_thin, vel_front, vel_back)

TypeError: calculate_vel() takes from 1 to 2 positional arguments but 7 were given